# 🌟 Waldiez Runner: Jupyter Notebook Example

This notebook demonstrates how to interact with the Waldiez Runner server using the client SDK. It covers:
- Authentication (via `clients-api` or `tasks-api`)
- Creating and managing tasks
- Listening to task output via WebSocket
- Sending input to running tasks
- Downloading results


## 🔐 Authentication

In [9]:
import os
import sys
from pathlib import Path

try:
    from waldiez_runner.client import (
        TasksClient,
        TaskStatus,
        ClientsAdmin,
        ClientCreateRequest,
        ClientCreateResponse,
        TaskCreateRequest,
        UserInputRequest,
    )
except ImportError:
    # Likely running from examples/jupyter — add project root to path
    sys.path.append(str(Path(".").resolve().parent.parent))
    from waldiez_runner.client import (
        TasksClient,
        TaskStatus,
        ClientsAdmin,
        ClientCreateRequest,
        ClientCreateResponse,
        TaskCreateRequest,
        UserInputRequest,
    )

# Base URL for the API
BASE_URL = os.environ.get("BASE_URL", "http://localhost:8000")

# Auth credentials from env or prompt
_CLIENT_ID = os.environ.get("CLIENT_ID") or input("CLIENT_ID: ")
_CLIENT_SECRET = os.environ.get("CLIENT_SECRET") or input("CLIENT_SECRET: ")
_AUDIENCE = os.environ.get("AUDIENCE") or input("AUDIENCE: ")
CLIENT_ID = _CLIENT_ID.strip()
CLIENT_SECRET = _CLIENT_SECRET.strip()
AUDIENCE = _AUDIENCE.strip()
os.environ["WALDIEZ_CLIENT_ID"] = CLIENT_ID
os.environ["WALDIEZ_CLIENT_SECRET"] = CLIENT_SECRET
os.environ["WALDIEZ_AUDIENCE"] = AUDIENCE

In [10]:
# Make sure we have a client with the right audience (tasks-api)
# if we create a new one, let's delete it at the end
CREATED_NEW_CLIENT = False

if AUDIENCE == "clients-api":
    admin = ClientsAdmin(
        on_error=lambda e: print(f"Error: {e}"),
        on_auth_error=lambda e: print(f"Auth Error: {e}"),
        # on_auth_token=lambda token: print(f"Auth Token: {token}"),
    )
    admin.configure(BASE_URL, CLIENT_ID, CLIENT_SECRET)

    print("Creating a new client with tasks-api audience...")
    client_create = ClientCreateRequest(
        # client_id=CLIENT_ID,  # not these, they are existing (for clients-api)
        # client_secret=CLIENT_SECRET,  # let's let the server generate them
        audience="tasks-api",
        description="Client for tasks-api",  # optional
    )
    client_info: ClientCreateResponse = admin.create_client(
        client_data=client_create
    )
    TASKS_CLIENT_ID = client_info.client_id
    TASKS_CLIENT_SECRET = client_info.client_secret
    CREATED_NEW_CLIENT = True
    print("Created tasks-api client.")
else:
    TASKS_CLIENT_ID = CLIENT_ID
    TASKS_CLIENT_SECRET = CLIENT_SECRET

Creating a new client with tasks-api audience...
Created tasks-api client.


In [11]:
# Now initialize the TasksClient
client = TasksClient()
client.configure(
    base_url=BASE_URL,
    client_id=TASKS_CLIENT_ID,
    client_secret=TASKS_CLIENT_SECRET,
    on_auth_token=None,  # we could use a custom handler here
    on_auth_error=None,  # and here
    on_error=None,  # and here
)
print("Authenticated as tasks-api client.")

Authenticated as tasks-api client.


## 🚫 Cancel any existing running tasks if any

In [12]:
current_tasks = client.list_tasks()
print(f"Current tasks: {current_tasks}")
for item in current_tasks.items:
    if item.status in (
        TaskStatus.RUNNING,
        TaskStatus.PENDING,
        TaskStatus.WAITING_FOR_INPUT,
    ):
        # Cancel the task if it's active (running, pending, or waiting for input)
        print(f"Cancelling task {item.id} with status {item.status}")
        client.cancel_task(item.id)

Current tasks: items=[] total=0 page=1 size=50 pages=0


## 📥 Submit a New Task

In [ ]:
# Upload a .waldiez flow
FLOW_PATH = os.path.join(".", "..", "dummy_with_input.waldiez")
with open(FLOW_PATH, "rb") as f:
    file_data = f.read()

task_create = TaskCreateRequest(
    file_data=file_data,
    file_name=FLOW_PATH.rsplit("/", maxsplit=1)[-1],
    input_timeout=60,
)
task = client.create_task(
    task_data=task_create,
)
print("📦 Task triggered:", task.id)

📦 Task created: 019639f3895f430a086fb01da307da1f


## 📡 Listen to Task Output

In [14]:
from queue import Queue

queue: Queue[str] = Queue()


def on_message(msg: str):
    print("📥", msg)
    queue.put(msg)


def on_error(err: str):
    print("❌ Error:", err)


client.start_ws_listener(
    task_id=task.id, on_message=on_message, on_error=on_error
)

📥 {"type":"status","timestamp":1744728983959486,"data":{"task_id":"019639f3895f430a086fb01da307da1f","status":"PENDING","created_at":"2025-04-15T14:56:23.903","updated_at":"2025-04-15T14:56:23.903","results":null,"input_request_id":null}}
📥 {"type":"print","data":"Installing requirements: pyautogen[openai]==0.8.6\n","task_id":"019639f3895f430a086fb01da307da1f","timestamp":"1744728990055849","id":"1744728990056-0"}
📥 {"type":"print","data":"Requirement already satisfied: pyautogen==0.8.6 in /home/user/.local/lib/python3.12/site-packages (from pyautogen[openai]==0.8.6) (0.8.6)\n","task_id":"019639f3895f430a086fb01da307da1f","timestamp":"1744728990542072","id":"1744728990542-0"}
📥 {"type":"print","data":"Requirement already satisfied: anyio<5.0.0,>=3.0.0 in /home/user/.local/lib/python3.12/site-packages (from pyautogen==0.8.6->pyautogen[openai]==0.8.6) (4.9.0)\n","task_id":"019639f3895f430a086fb01da307da1f","timestamp":"1744728990552853","id":"1744728990552-0"}
📥 {"type":"print","data":"R

WebSocket Error (sync): did not receive a valid HTTP response, reconnecting in 2 seconds...
WebSocket Error (sync): [Errno 61] Connection refused, reconnecting in 4 seconds...
WebSocket Error (sync): [Errno 61] Connection refused, reconnecting in 8 seconds...
WebSocket Error (sync): [Errno 61] Connection refused, reconnecting in 16 seconds...
WebSocket Error (sync): [Errno 61] Connection refused, reconnecting in 30 seconds...
Stopping listener due to stop event.


❌ Error: [Errno 61] Connection refused


## 🎤 Send Input (if prompted)

In [15]:
def send_input(request_id: str, data: str):
    input_msg = UserInputRequest(
        task_id=task.id,
        request_id=request_id,
        data=data,
    )
    # alternatively, as plain dict:
    # input_msg = {
    #     "task_id": task.id,
    #     "request_id": request_id,
    #     "data": data,
    # }
    client.send_user_input(request_data=input_msg)

## ⏳ Wait for Completion and Download Results

In [16]:
import time

while True:
    current = client.get_task_status(task_id=task.id)
    if current.status in (
        TaskStatus.COMPLETED,
        TaskStatus.FAILED,
        TaskStatus.CANCELLED,
    ):
        print("Finished:", current.status)
        break
    print("⏳ Status:", current.status)
    if current.status == TaskStatus.WAITING_FOR_INPUT:
        send_input(request_id=current.input_request_id, data="Hello world!")
    time.sleep(2)

zip_data = client.download_task_results(task_id=task.id)
with open("results.zip", "wb") as f:
    f.write(zip_data)
print("📁 Results saved to results.zip")

⏳ Status: TaskStatus.PENDING
⏳ Status: TaskStatus.PENDING
⏳ Status: TaskStatus.PENDING
⏳ Status: TaskStatus.PENDING
⏳ Status: TaskStatus.WAITING_FOR_INPUT
⏳ Status: TaskStatus.WAITING_FOR_INPUT
Finished: TaskStatus.COMPLETED
📁 Results saved to results.zip


In [17]:
import os
import zipfile

# Unzip the results
with zipfile.ZipFile("results.zip", "r") as zip_ref:
    zip_ref.extractall("results")
print("📂 Unzipped results to results")
for root, dirs, files in os.walk("results"):
    for file in files:
        print(f"📄 {file}")

📂 Unzipped results to results
📄 dummy_with_input.waldiez
📄 waldiez_flow_api_keys.py
📄 dummy_with_input.py
📄 Waldiez flow.mmd
📄 flow.db
📄 oai_wrappers.csv
📄 oai_clients.csv
📄 version.json
📄 function_calls.csv
📄 events.json
📄 chat_completions.json
📄 version.csv
📄 agents.json
📄 function_calls.json
📄 oai_wrappers.json
📄 oai_clients.json
📄 chat_completions.csv
📄 agents.csv
📄 events.csv


## 🗑️ Cleanup

In [18]:
import shutil
import os

# Clean up the results
if os.path.exists("results"):
    shutil.rmtree("results")
    print("🗑️ Deleted results directory.")

if os.path.exists("results.zip"):
    os.remove("results.zip")
    print("🗑️ Deleted results.zip")

if CREATED_NEW_CLIENT:
    clients_admin = ClientsAdmin()
    clients_admin.configure(BASE_URL, CLIENT_ID, CLIENT_SECRET)
    clients_admin.delete_client(client_id=TASKS_CLIENT_ID)
    print("🗑️ Deleted tasks-api client.")

os.environ.pop("WALDIEZ_CLIENT_ID", None)
os.environ.pop("WALDIEZ_CLIENT_SECRET", None)
os.environ.pop("WALDIEZ_AUDIENCE", None)
print("🗑️ Deleted environment variables.")

🗑️ Deleted results directory.
🗑️ Deleted results.zip
🗑️ Deleted tasks-api client.
🗑️ Deleted environment variables.
